# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects

In [2]:
import datetime
import os
import sys

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool, MessageRole, ListSortOrder
from dotenv import load_dotenv

In [3]:
load_dotenv("azure.env")

True

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 09-Jun-2025 16:30:29


## 1. Grounding Bing Search

### Connect to AI Foundry

In [32]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to bing grounding service

In [33]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("AZURE_BING_CONNECTION_ID")

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

### Create agent with Bing Grounding tool

In [34]:
# Create the agent with the Bing grounding tool
instructions = "You are an AI Agent that can do some web search using Bing"
agent = project_client.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-bing-grounding-agent",
        instructions=instructions,
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    # [END create_agent_with_bing_grounding_tool]

print(f"Created agent, ID: {agent.id}")

bing
bing.definitions


Created agent, ID: asst_sO5reTYCQVM5QUQ3M7SaL5IB


[{'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/Microsoft.CognitiveServices/accounts/aq-ai-foundry-Sweden-Central/projects/firstProject/connections/aqbinggrounding002', 'market': '', 'set_lang': '', 'count': 5, 'freshness': ''}]}}]

### Create THREAD

In [35]:
# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_Lc9e9MfB1Vs28lhrY5ZAEyWa


### Query #1

In [36]:
query = "What is the current price of MICROSOFT stock?"

### Create first message in the thread

In [37]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_XtV8U3WVPTT5hATzXw23cmBW


### Create first RUN

In [38]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


### Check status of the RUN and tool calls

In [39]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage


Step step_d3mEeaco4fgQitd1fW5IbM2k status: completed
Step step_EJIHUWYoTM8rSlKB2n6XzZH3 status: completed
  Tool calls:
    Tool Call ID: call_TlttQXlKg5HPgVekRFhL4UCd
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=search(query: "current price of Microsoft stock")



{'prompt_tokens': 1270, 'completion_tokens': 150, 'total_tokens': 1420, 'prompt_token_details': {'cached_tokens': 0}}

### Get response

In [40]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: The current price of Microsoft (MSFT) stock can be checked in real-time on financial platforms such as Yahoo Finance, Google Finance, or Nasdaq. For the most up-to-date value, visit one of these sources, as stock prices fluctuate continuously throughout trading hours. As of now, Microsoft stock has recently opened at a fresh all-time high, reflecting considerable investor interest【3:0†Microsoft Corporation (MSFT) Stock Price, News, Quote & History - Yahoo ...】【3:2†Microsoft Corp (MSFT) Stock Price & News - Google Finance】. For a specific real-time value, please check Yahoo Finance or Google Finance.
URL Citation: [Microsoft Corporation (MSFT) Stock Price, News, Quote & History - Yahoo ...](https://finance.yahoo.com/quote/MSFT/)
URL Citation: [Microsoft Corp (MSFT) Stock Price & News - Google Finance](https://www.google.com/finance/quote/MSFT%3ANASDAQ)


### Example #2

In [41]:
query = "find out about the latest iPhone"

In [42]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_9hxDwDCV30vdLjc4MvNBlFIe


In [43]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [44]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_kBneE6oEiMqoqviJA47FfUh7 status: completed
Step step_9P78GpELehNd03IHAR8Ob1ml status: completed
  Tool calls:
    Tool Call ID: call_mo0eB38zA6LXDzO8iN5K5Dl6
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=search(query: "latest iPhone 2025")



{'prompt_tokens': 1735, 'completion_tokens': 156, 'total_tokens': 1891, 'prompt_token_details': {'cached_tokens': 0}}

In [45]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: The latest iPhone lineup in 2025 features several new models, including the iPhone 16, iPhone 16 Pro, and a new budget-friendly iPhone SE 4. These phones have notable improvements in camera technology, battery life, and design. There are also rumors about an "iPhone Air" with a larger 6.55 to 6.65-inch screen for those preferring bigger displays. This year's iPhones offer options tailored to a variety of users, from affordable models to advanced Pro versions with the latest features【7:0†Geeky Gadgets】【7:2†CNET】【7:4†mobilen.nu】.
URL Citation: [2025 iPhone Lineup: Features, Specs, and Pricing - Geeky Gadgets](https://www.geeky-gadgets.com/2025-iphones/)
URL Citation: [Best iPhone in 2025: Here's Which Apple Phone You Should Buy](https://www.cnet.com/tech/mobile/best-iphone/)
URL Citation: [Apples iPhone 2025: Större skärmar, förbättrade funktioner och ...](https://mobilen.nu/iphone-2025-storre-skarmar-foerbattrade-funktioner/)


### Example #3

In [20]:
query = "What is the weather informations for Mexico city? Show the report and use emojis and be funny"

In [21]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_ighLZrI3BZx4nSB0Fp8O3ETw


In [22]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [23]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_76BbPlQehCGSfbBdVll3zSLD status: completed
Step step_wlR28Qxbf6iZi93bXmamc2Hw status: completed
  Tool calls:
    Tool Call ID: call_1SuouAcIDjHyVLNXR653g4cJ
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=search(query: "current weather in Mexico City")



{'prompt_tokens': 1735, 'completion_tokens': 217, 'total_tokens': 1952, 'prompt_token_details': {'cached_tokens': 0}}

In [24]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: 🌞 Mexico City Weather Report! 🇲🇽

Right now, Mexico City is rocking that comfortable weather: mostly sunny with a bit of cloud cover later—perfect for tacos outdoors! 🌮😎

- Temperature: Comfortable (neither sweat nor shiver weather!) 🌡️
- Skies: Mostly sunny with some playful clouds rolling in later. ☀️➡️⛅
- Air: Good for everyone, unless you’re a cloud (then, sorry, today’s not your day)! 🌬️
- Recommendation: Grab your sunscreen, shades, and maybe a sombrero for style. Just don’t forget your sense of humor! 😂🕶️

Conclusion: If you’re in Mexico City, the weather is as friendly as a mariachi at a fiesta!
For the freshest details, check your favorite weather app because the sky likes to surprise! 【11:1†Weather.com】【11:0†Accuweather】
URL Citation: [Weather Forecast and Conditions for Mexico City, Mexico - The Weather ...](https://weather.com/weather/today/l/6121681b2c5df01145b9723d497c595c53ae08104787aa1c26bafdf2fb875c07)
URL Citation: [Mexico City, México City, Mexico Cur

### Post processing

In [25]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_y4ud6cPMKlrX8IkC7GV6Qc2q, Name: my-bing-grounding-agent, Model: gpt-4.1, Instructions: You are a helpful agent


In [27]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")

In [28]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")